<a href="https://colab.research.google.com/github/13izay/Classificationcomparison/blob/main/cash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

TensorFlow 2.x selected.


In [ ]:
import sklearn
from sklearn import metrics
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator

import os

Using TensorFlow backend.


In [ ]:
URL_="https://storage.googleapis.com/cashrecog/cash_data.zip"
path_to_zip= keras.utils.get_file("cash_data.zip", origin=URL_, extract=True, cache_dir="/content/", cache_subdir="datasets_cash" )
PATH= os.path.join(os.path.dirname(path_to_zip),"cash_data" )


2492047360/2492039499 [==============================] - 34s 0us/step


In [ ]:
train_dir= os.path.join(PATH, "train")
valid_dir= os.path.join(PATH, "valid")

Epoch=10
IMG_HEIGHT=150
IMG_WIDTH=150
Batch_size=100

In [ ]:
train_image_generator= ImageDataGenerator(rescale=1./255,
                                          height_shift_range=.15,
                                          width_shift_range=.15,
                                          zoom_range=0.5,
                                          vertical_flip=True,
                                          rotation_range=45)

valid_image_generator= ImageDataGenerator(rescale=1./255,
                                          height_shift_range=.15,
                                          width_shift_range=.15,
                                          zoom_range=0.5,
                                          vertical_flip=True,
                                          rotation_range=45)


In [ ]:
train_data= train_image_generator.flow_from_directory(train_dir,
                                                      target_size=[IMG_HEIGHT,IMG_WIDTH],
                                                      shuffle=True,
                                                      batch_size=Batch_size)
valid_data= train_image_generator.flow_from_directory(valid_dir,
                                                      target_size=[IMG_HEIGHT,IMG_WIDTH],
                                                      shuffle=True,
                                                      batch_size=Batch_size)


Found 10500 images belonging to 7 classes.
Found 3500 images belonging to 7 classes.


In [ ]:
pre_model=keras.applications.VGG16(input_shape=[IMG_HEIGHT,IMG_WIDTH,3], include_top=False, weights="imagenet")

for i, layer in enumerate(pre_model.layers):
  print(i, layer.name)

for layer in pre_model.layers[:6]:
  layer.trainable=True


58892288/58889256 [==============================] - 6s 0us/step
0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool


In [ ]:
pre_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [ ]:
model= tf.keras.models.Sequential([
    pre_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(7, activation="softmax")
])




In [ ]:
model.compile(keras.optimizers.Adam(learning_rate=0.001),
              keras.losses.BinaryCrossentropy(),
              metrics=["accuracy"])



In [ ]:
model.fit_generator(
    train_data,
    validation_data=valid_data,
    epochs=Epoch
)


Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 105 steps, validate for 35 steps
Epoch 1/10
105/105 [==============================] - 246s 2s/step - loss: 0.4114 - accuracy: 0.8554 - val_loss: 0.3834 - val_accuracy: 0.8571
Epoch 2/10
105/105 [==============================] - 232s 2s/step - loss: 0.3621 - accuracy: 0.8619 - val_loss: 0.3289 - val_accuracy: 0.8673
Epoch 3/10
105/105 [==============================] - 230s 2s/step - loss: 0.3084 - accuracy: 0.8710 - val_loss: 0.3074 - val_accuracy: 0.8675
Epoch 4/10
105/105 [==============================] - 228s 2s/step - loss: 0.2378 - accuracy: 0.8931 - val_loss: 0.2036 - val_accuracy: 0.9018
Epoch 5/10
105/105 [==============================] - 219s 2s/step - loss: 0.1738 - accuracy: 0.9233 - val_loss: 0.1965 - val_accuracy: 0.9158
Epoch 6/10
105/105 [==============================] - 213s 2s/step - loss: 0.1455 - accuracy: 0.9419 - val_loss: 0.1

In [ ]:
save_path="/content/drive/My Drive/cash_model.h5"
model.save(save_path)

In [ ]:
weights_path="/content/drive/My Drive/cash_weights.ckpt"
model.save_weights(weights_path)